In [8]:
n_vocab = 1024
model_file = f'models/bpe{n_vocab}wu.pack'

In [12]:
from os.path import expanduser
from tqdm import tqdm

def data_iter(chunk=1e5):
    fname = expanduser('~/dev/data/text/wiki/zim/wikipedia_en_all_nopic_2024-04.zim')
    data = b''
    for page in zimloader.read_zims([fname], nthreads=16):
        data += bytes(page, 'utf8')
        if len(data) > chunk:
            yield data
            data = b''


# def data_iter(chunk=1e6):
#     fname = expanduser('~/dev/data/text/wiki/zim/wikipedia_en_all_nopic_2024-04.shuf.txt')
#     data = b''
#     while True:
#         with open(fname, 'rb') as f:
#             for line in f:
#                 line = line.trim()
#                 if len(line) < 100:
#                     continue
#                 data += bytes(line, 'utf8')
#                 if len(data) > chunk:
#                     yield data
#                     data = b''

# def data_iter(chunk=1e6):
#     fname = expanduser('~/dev/data/text/wiki/zim/wikipedia_en_all_nopic_2024-04.shuf.txt')
#     # data = b''
#     while True:
#         with open(fname, 'rb') as f:
#             while (b := f.read(int(chunk))):
#                 yield b

# fname = expanduser('~/dev/data/text/wiki/zim/wikipedia_en_all_nopic_2024-04.shuf.txt')
# data = b''
# with open(fname, 'r') as f:
#     for line in (prog := tqdm(f)):
#         line = line.strip()
#         if len(line) < 100:
#             continue
#         data += bytes(line, 'utf8')
#         prog.set_description(f'{len(data)}')
#         if len(data) > 1e7:
#             break

data = next(data_iter(1e7))

In [13]:
from sqrll import zimloader
from tqdm import tqdm
import btok

btr = btok.Trainer(n_vocab, whitespace_unigram=True)

def traingen():
    for i in range(btr.num_tokens()):
        yield i
    while btr.update(data, tokens_to_add=1):
        yield btr.num_tokens()

for _ in (prog := tqdm(traingen())):
    prog.refresh()

# for chunk in (prog := tqdm(data_iter(1e7), unit='tok', total=n_vocab)):
#     btr.update(chunk, tokens_to_add=1)
#     n = btr.num_tokens()
#     prog.update(n - prog.n)
#     prog.refresh()
#     if n == n_vocab:
#         prog.close()
#         break

# for chunk in data_iter():
#     print(len(chunk))
#     btr.update(chunk, tokens_to_add=1)
#     print(btr.num_tokens())
#     if btr.num_tokens() == n_vocab:
#         break

with open(model_file, 'wb') as f:
    f.write(btr.tokenizer.pack())

1024it [00:45, 22.73it/s] 


In [ ]:
# model_file = f'models/bpe{n_vocab}w2.pack'

with open(model_file, 'rb') as f:
    btz = btok.Tokenizer(f.read())

for i in range(btz.num_tokens()):
    print(btz.token(i))

In [15]:
# data = next(data_iter(1e7))
toks = btz.encode(data)
len(toks)

5479833

In [16]:
import numpy as np

orig_bits = len(data) * 8
new_bits = len(toks) * np.log2(btz.num_tokens())

new_bits / orig_bits

0.6848744761800397

In [7]:
import torch

torch.nn.functional.sigmoid(torch.tensor([5]))

tensor([0.9933])